https://public.roboflow.com/classification   
roboflow -> format: coco -> terminal 복사(앞에 ! 붙이기)  
Class : dandelion, daisy

In [ ]:
# 학습 중간에 메모리 정리 ("사용가능한 RAM을 모두 사용한 후 세션이 다운되었습니다" 방지)
import torch
torch.cuda.empty_cache()

In [1]:
# 데이터 불러오기: https://public.roboflow.com/classification/flowers_classification/3 -> OpenAI, show download code
!curl -L "https://public.roboflow.com/ds/kq0jhQKygf?key=3q91k5th3G" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/daisy/4613992315_143ccc2a10_m_jpg.rf.c97783768121e29070fd6dad98707437.jpg  
 extracting: train/daisy/4654579740_6671a53627_m_jpg.rf.1778ca1c7044256b0e21f0109fa78621.jpg  
 extracting: train/daisy/4654579740_6671a53627_m_jpg.rf.6332dd39a680e80f7489bea5f991c9ce.jpg  
 extracting: train/daisy/4654579740_6671a53627_m_jpg.rf.840413b8f71912e8b467aab8060599c8.jpg  
 extracting: train/daisy/4654579740_6671a53627_m_jpg.rf.85731e739e4083853ad0a3bd2ba2708a.jpg  
 extracting: train/daisy/4654579740_6671a53627_m_jpg.rf.b2d1a29a941b34de666e45c866c0e322.jpg  
 extracting: train/daisy/4657354814_f368762c53_m_jpg.rf.68a86b8b763bae2dd158b0a17d302775.jpg  
 extracting: train/daisy/4657354814_f368762c53_m_jpg.rf.a58c0e5914f924a1fd65a00669a0b298.jpg  
 extracting: train/daisy/4657354814_f368762c53_m_jpg.rf.a75798ec677ef2690d7f539b231a775c.jpg  
 extracting: train/daisy/4657354814_f368762c53_m_jpg.rf.aaffdf2b9c213cf8ad6be8bff1229f75.jpg  
 extracting: t

In [ ]:
ls

README.dataset.txt  README.roboflow.txt  sample_data/  test/  train/  valid/


In [ ]:
cd sample_data

/content/sample_data


In [2]:
import torch

import torch.nn as nn

class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.feature = nn.Sequential(
            # 1-layer
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 2-layer
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 3-layer
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 4-layer
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 5-layer
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2),
            nn.Dropout(p=0.5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.feature(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [3]:
import glob
import os
import numpy as np
from PIL import Image


from torchvision import transforms
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

def dataloader(data_pair):
    # 데이터셋과 라벨 분리
    images, labels = zip(*data_pair)  # 이미지와 라벨을 분리

    # 데이터셋 분할
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels
    )
    vali_images, test_images, vali_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=0.5, stratify=temp_labels
    )

    # DataLoader 생성
    train = DataLoader(list(zip(train_images, train_labels)), batch_size=16, shuffle=True)
    vali = DataLoader(list(zip(vali_images, vali_labels)), batch_size=16, shuffle=True)
    test = DataLoader(list(zip(test_images, test_labels)), batch_size=16, shuffle=False)

    return train, vali, test

def datatransfer():
    image_path = sorted(glob.glob("/content/train/*/*.jpg"))  # 정렬하여 순서 일치 보장
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    data_pair = []

    # 클래스 이름을 고유 ID로 매핑
    class_names = sorted({os.path.basename(os.path.dirname(p)) for p in image_path})
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}

    for image in image_path:
        # 이미지 처리
        img = Image.open(image)
        img = transform(img)

        # 라벨 처리
        label_name = os.path.basename(os.path.dirname(image))
        label_data = class_to_idx[label_name]

        # 데이터 쌍 생성
        data_pair.append((img, label_data))

    # DataLoader 반환
    return dataloader(data_pair)

### 단순 학습

In [ ]:
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = VGGNet().to(device)

learning_rate = 0.001
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train, vali, test = datatransfer()

# training
train_loss = []
vali_loss = []
vali_best_acc = []

train_start = time.time()
for epoch in range(epochs):
    epoch_start = time.time()
    for image, label in train:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        print(f"epoch: {epoch}, train loss: {loss.item()}")

    # validation
    for image, label in vali:
        image = image.to(device)
        label = label.to(device)

        output = model(image)
        loss = criterion(output, label)
        vali_loss.append(loss.item())
        print(f"epoch: {epoch}, vali loss: {loss.item()}")
    epoch_end = time.time()
    print(f"epoch: {epoch}, time: {epoch_end - epoch_start}")

    # model save
    if epoch == 0:
        torch.save(model.state_dict(), "model.pt")

train_end = time.time()
print(f"train time: {train_end - train_start}")

cuda
epoch: 0, train loss: 0.6935462951660156
epoch: 0, train loss: 0.7597917318344116
epoch: 0, train loss: 0.6194759607315063
epoch: 0, train loss: 0.7594903707504272
epoch: 0, train loss: 0.7594903111457825
epoch: 0, train loss: 0.8844903707504272
epoch: 0, train loss: 0.8457331657409668
epoch: 0, train loss: 0.5719903707504272
epoch: 0, train loss: 0.8932188749313354
epoch: 0, train loss: 0.9944759607315063
epoch: 0, train loss: 0.9469903707504272
epoch: 0, train loss: 0.6107475161552429
epoch: 0, train loss: 1.0094903707504272
epoch: 0, train loss: 0.7594903707504272
epoch: 0, train loss: 0.666961669921875
epoch: 0, train loss: 0.8457331657409668
epoch: 0, train loss: 0.6582331657409668
epoch: 0, train loss: 0.854461669921875
epoch: 0, train loss: 0.7594903111457825
epoch: 0, train loss: 0.7594903111457825
epoch: 0, train loss: 0.6194760799407959
epoch: 0, train loss: 0.7207332253456116
epoch: 0, train loss: 0.6969903111457825
epoch: 0, train loss: 0.6969903111457825
epoch: 0, tra

### + accuracy, f1 score
- Final Validation Accuracy: 0.5856
- Final Validation F1-score: 0.4325

In [4]:
import time
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = VGGNet().to(device)

learning_rate = 0.001
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train, vali, test = datatransfer()

# Training and validation variables
train_loss = []
vali_loss = []

# For calculating accuracy and F1 score after training
all_preds = []
all_labels = []

train_start = time.time()
for epoch in range(epochs):
    epoch_start = time.time()

    # Training phase
    model.train()
    for image, label in train:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        print(f"epoch: {epoch}, train loss: {loss.item()}")

    # Validation phase
    model.eval()
    for image, label in vali:
        image = image.to(device)
        label = label.to(device)

        output = model(image)
        loss = criterion(output, label)
        vali_loss.append(loss.item())

        # Store predictions and labels for accuracy and F1 calculation
        _, predicted = torch.max(output, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

        print(f"epoch: {epoch}, vali loss: {loss.item()}")

    epoch_end = time.time()
    print(f"epoch: {epoch}, time: {epoch_end - epoch_start}")

    # Save model after first epoch
    if epoch == 0:
        torch.save(model.state_dict(), "model.pt")

train_end = time.time()
print(f"Train time: {train_end - train_start}")

# After training is complete, calculate accuracy and F1-score
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')  # 'weighted' to account for imbalanced classes

print(f"Final Validation Accuracy: {accuracy:.4f}")
print(f"Final Validation F1-score: {f1:.4f}")

cuda
epoch: 0, train loss: 0.6935279369354248
epoch: 0, train loss: 0.7649291157722473
epoch: 0, train loss: 0.8312283158302307
epoch: 0, train loss: 0.7594903707504272
epoch: 0, train loss: 0.9707331657409668
epoch: 0, train loss: 0.7207332253456116
epoch: 0, train loss: 1.0094903707504272
epoch: 0, train loss: 0.9232475161552429
epoch: 0, train loss: 0.8544617295265198
epoch: 0, train loss: 0.7207332849502563
epoch: 0, train loss: 0.9944761395454407
epoch: 0, train loss: 0.9707332253456116
epoch: 0, train loss: 0.8307188153266907
epoch: 0, train loss: 0.884490430355072
epoch: 0, train loss: 0.8219903707504272
epoch: 0, train loss: 0.7444760799407959
epoch: 0, train loss: 0.729461669921875
epoch: 0, train loss: 0.8219903707504272
epoch: 0, train loss: 0.9319760799407959
epoch: 0, train loss: 0.7832332253456116
epoch: 0, train loss: 0.8607475161552429
epoch: 0, train loss: 0.6582332849502563
epoch: 0, train loss: 0.7769473791122437
epoch: 0, train loss: 0.7682188749313354
epoch: 0, tra

### + Early Stopping
- Test Accuracy: 0.4154

In [5]:
import torch

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, path="best_model.pth"):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        # 처음에 best_loss를 설정
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_model(model)
        # 검증 손실이 개선되지 않으면 카운트 증가
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.save_model(model)
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def save_model(self, model):
        self.best_model_wts = model.state_dict()
        torch.save(self.best_model_wts, self.path)

In [6]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = VGGNet().to(device)

learning_rate = 0.001
epochs = 50  # 최대 에폭 수
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train, vali, test = datatransfer()

# EarlyStopping 초기화
early_stopping = EarlyStopping(patience=5, min_delta=0.001, path="best_model.pth")

train_loss = []
vali_loss = []

train_start = time.time()

for epoch in range(epochs):
    epoch_start = time.time()

    # Training phase
    model.train()
    running_train_loss = 0.0
    for image, label in train:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train)
    train_loss.append(avg_train_loss)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for image, label in vali:
            image = image.to(device)
            label = label.to(device)

            output = model(image)
            loss = criterion(output, label)
            running_val_loss += loss.item()

    avg_val_loss = running_val_loss / len(vali)
    vali_loss.append(avg_val_loss)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

    # EarlyStopping 적용
    early_stopping(avg_val_loss, model)

    # Early stopping을 활성화한 경우, 학습을 종료합니다.
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    epoch_end = time.time()
    print(f"Epoch Time: {epoch_end - epoch_start:.2f}s")

train_end = time.time()
print(f"Training complete in: {train_end - train_start:.2f}s")

# 가장 좋은 모델 로드
model.load_state_dict(early_stopping.best_model_wts)

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for image, label in test:
        image = image.to(device)
        label = label.to(device)

        output = model(image)
        _, predicted = torch.max(output, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")

cuda
Epoch 1/50, Train Loss: 0.8052
Epoch 1/50, Validation Loss: 0.8990
Epoch Time: 92.91s
Epoch 2/50, Train Loss: 0.7997
Epoch 2/50, Validation Loss: 0.8990
Epoch Time: 81.38s
Epoch 3/50, Train Loss: 0.7940
Epoch 3/50, Validation Loss: 0.8986
Epoch Time: 81.24s
Epoch 4/50, Train Loss: 0.8048
Epoch 4/50, Validation Loss: 0.8986
Epoch Time: 81.17s
Epoch 5/50, Train Loss: 0.8017
Epoch 5/50, Validation Loss: 0.8993
Epoch Time: 80.91s
Epoch 6/50, Train Loss: 0.8101
Epoch 6/50, Validation Loss: 0.8990
Early stopping at epoch 6
Training complete in: 498.53s
Test Accuracy: 0.4154


### + StepLR
- Accuracy: 0.4144, F1-score: 0.2429

In [ ]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# 모델과 optimizer 설정
model = VGGNet().to(device)
optimizer = Adam(model.parameters(), lr=0.001)

learning_rate = 0.001
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train, vali, test = datatransfer()

# StepLR 설정
step_lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # 10 에폭마다 학습률을 0.1배로 감소

# 학습 루프
for epoch in range(epochs):
    model.train()
    running_train_loss = 0.0
    for image, label in train:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # StepLR 학습률 스케줄러 업데이트
    step_lr_scheduler.step()  # 학습률 감소

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for image, label in vali:
            image = image.to(device)
            label = label.to(device)

            output = model(image)
            loss = criterion(output, label)
            running_val_loss += loss.item()

    avg_val_loss = running_val_loss / len(vali)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

cuda


In [4]:
# 평가 지표 추가
from sklearn.metrics import accuracy_score, f1_score
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import time
import matplotlib.pyplot as plt
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# 모델과 optimizer 설정
model = VGGNet().to(device)
optimizer = Adam(model.parameters(), lr=1e-4) #0.001

learning_rate = 0.001
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train, vali, test = datatransfer()

# StepLR 설정
step_lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # 10 에폭마다 학습률을 0.1배로 감소

# 학습 루프
for epoch in range(epochs):
    model.train()
    running_train_loss = 0.0
    for image, label in train:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # StepLR 학습률 스케줄러 업데이트
    step_lr_scheduler.step()  # 학습률 감소

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for image, label in vali:
            image = image.to(device)
            label = label.to(device)

            output = model(image)
            loss = criterion(output, label)
            running_val_loss += loss.item()

            # 예측값 생성 (Softmax 후 argmax로 클래스 예측)
            preds = torch.argmax(output, dim=1)

            # 예측값과 실제값 저장
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    # 평균 Validation 손실 계산
    avg_val_loss = running_val_loss / len(vali)

    # Accuracy와 F1-score 계산
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')  # weighted로 F1-score를 클래스 불균형에 맞게 계산

    # 결과 출력
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")

cuda
Epoch 1/10, Train Loss: 0.8427
Epoch 1/10, Validation Loss: 0.9000, Accuracy: 0.4144, F1-score: 0.2429
Epoch 2/10, Train Loss: 0.8506
Epoch 2/10, Validation Loss: 0.8986, Accuracy: 0.4144, F1-score: 0.2429
Epoch 3/10, Train Loss: 0.8439
Epoch 3/10, Validation Loss: 0.8982, Accuracy: 0.4144, F1-score: 0.2429
Epoch 4/10, Train Loss: 0.8505
Epoch 4/10, Validation Loss: 0.8990, Accuracy: 0.4144, F1-score: 0.2429
Epoch 5/10, Train Loss: 0.8480
Epoch 5/10, Validation Loss: 0.8990, Accuracy: 0.4144, F1-score: 0.2429
Epoch 6/10, Train Loss: 0.8521
Epoch 6/10, Validation Loss: 0.8986, Accuracy: 0.4144, F1-score: 0.2429
Epoch 7/10, Train Loss: 0.8432
Epoch 7/10, Validation Loss: 0.8979, Accuracy: 0.4144, F1-score: 0.2429
Epoch 8/10, Train Loss: 0.8443
Epoch 8/10, Validation Loss: 0.8997, Accuracy: 0.4144, F1-score: 0.2429
Epoch 9/10, Train Loss: 0.8437
Epoch 9/10, Validation Loss: 0.8986, Accuracy: 0.4144, F1-score: 0.2429
Epoch 10/10, Train Loss: 0.8452
Epoch 10/10, Validation Loss: 0.8990

In [ ]:
print(output)
print(label.min(), label.max())  # 라벨 범위 확인
for image, label in train:
    print(image.shape, label.shape)  # 이미지와 라벨 배치 크기 확인
    break
print(all_preds[:5], all_labels[:5])  # 예측 및 라벨 확인

In [ ]:
# 초기화
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)

model.apply(weights_init)

# 학습률 스케줄러 확인
step_lr_scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

### + Early Stopping & StepLR
- Accuracy: 0.5856, F1-score: 0.4325

In [5]:
import torch

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, path="best_model.pth"):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        # 처음에 best_loss를 설정
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_model(model)
        # 검증 손실이 개선되지 않으면 카운트 증가
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.save_model(model)
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def save_model(self, model):
        self.best_model_wts = model.state_dict()
        torch.save(self.best_model_wts, self.path)

In [6]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# 모델과 optimizer 설정
model = VGGNet().to(device)
optimizer = Adam(model.parameters(), lr=0.001)

learning_rate = 0.001
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train, vali, test = datatransfer()

# StepLR 설정
step_lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# EarlyStopping 설정 (예: patience=5, min_delta=0.001)
early_stopping = EarlyStopping(patience=5, min_delta=0.001, path="best_model.pth")

from sklearn.metrics import accuracy_score, f1_score

# 학습 루프
for epoch in range(epochs):
    model.train()
    running_train_loss = 0.0
    for image, label in train:
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # StepLR 학습률 스케줄러 업데이트
    step_lr_scheduler.step()

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for image, label in vali:
            image = image.to(device)
            label = label.to(device)

            output = model(image)
            loss = criterion(output, label)
            running_val_loss += loss.item()

            # 예측값 생성
            preds = torch.argmax(output, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    # 평균 Validation 손실 계산
    avg_val_loss = running_val_loss / len(vali)

    # Accuracy와 F1-score 계산
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')  # weighted F1-score

    # 결과 출력
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, "
          f"Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")

    # EarlyStopping 적용
    early_stopping(avg_val_loss, model)

    # Early stopping 조건 만족 시 학습 종료
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch + 1}")
        break

cuda
Epoch 1/10, Train Loss: 0.7590
Epoch 1/10, Validation Loss: 0.7272, Accuracy: 0.5856, F1-score: 0.4325
Epoch 2/10, Train Loss: 0.7616
Epoch 2/10, Validation Loss: 0.7283, Accuracy: 0.5856, F1-score: 0.4325
Epoch 3/10, Train Loss: 0.7575
Epoch 3/10, Validation Loss: 0.7283, Accuracy: 0.5856, F1-score: 0.4325
Epoch 4/10, Train Loss: 0.7584
Epoch 4/10, Validation Loss: 0.7283, Accuracy: 0.5856, F1-score: 0.4325
Epoch 5/10, Train Loss: 0.7587
Epoch 5/10, Validation Loss: 0.7286, Accuracy: 0.5856, F1-score: 0.4325
Epoch 6/10, Train Loss: 0.7507
Epoch 6/10, Validation Loss: 0.7268, Accuracy: 0.5856, F1-score: 0.4325
Early stopping at epoch 6
